параметры

In [44]:
tests = [
    {
        'p': [0.5, 0.5],
        'a': [[0.8, 0.2], [0.2, 0.8]],
        'b': [[0.5, 0.5], [0.1, 0.9]],
        'seq_open': 'ОРОРОРООРРРРРРРРРРОООООООО',
    },
    {
        'p': [0.5, 0.5],
        'a': [[0.5, 0.5], [0.5, 0.5]],
        'b': [[0.5, 0.5], [0.51, 0.49]],
        'seq_open': 'ОРОРОРООРРРРРРРРРРОООООООО',
    },
]

Витерби

In [53]:
def viterbi(p, a, b, seq):
    seq_open = list(map(lambda x: 0 if x == 'О' else 1, seq))
    head, *tail = seq_open
    n = len(p)
    m = len(seq_open)
    d_cur = p.copy()
    for i in range(n):
        d_cur[i] *= b[i][head]
    states = []
    for t in range(1, m):
        d_next = []
        s = []
        for j in range(n):
            i_max = 0
            d_max = 0
            for i in range(n):
                d = d_cur[i]*a[i][j]*b[j][seq_open[t]]
                if d > d_max:
                    d_max = d
                    i_max = i
            d_next.append(d_max)
            s.append(i_max)
        d_cur = d_next.copy()
        states.append(s)
    i_max_last = 0
    for i in range(1, n):
        if d_cur[i] > d_cur[i_max_last]:
            i_max_last = i
    seq_hidden = str(i_max_last+1)

    # восстанавливаем последовательность скрытых состояний  
    for t in range(m-2, -1, -1):
        i_max_last = states[t][i_max_last]
        seq_hidden = str(i_max_last+1)+seq_hidden
#     print(states)
#     print(i_max_last)
#     print(''.join(list(map(lambda x: str(x), seq_open))))
    return seq_hidden

Forward-Backward

In [120]:
def forward_backward(p, a, b, seq):
    seq_open = list(map(lambda x: 0 if x == 'О' else 1, seq))
    n = len(p)
    m = len(seq_open)
    
    # инициализируем альфы
    alphas = list(map(
        lambda x: [x],
        product(p, column(b, seq_open[0]))
    ))
    
    # вычисляем альфы
    for t in range(1, m):
        for j in range(n):
#             print(j, column(alphas, t-1), column(a, j))
            alphas[j].append(b[j][seq_open[t]] * dot(column(alphas, t-1), column(a, j)))
    
    # инициализируем беты
    betas = []
    for _ in range(n):
        betas.append([1])
    
    # вычисляем беты
    for t in range(m-1, 0, -1):
        last_betas = column(betas, -1)
        for i in range(n):
            betas[i].append(dot(
                product(last_betas, a[i]),
                column(b, seq_open[t])
            ))
    for i in range(n):
        betas[i].reverse()
    
    # вычисляем нормирующий множитель
    prob = sum(column(alphas, -1))
    
#     print(prob)
#     print(betas)
    
    # вычисляем и выводим результат
#     results = []
    for i in range(n):
        result = [ (a/prob)*b for (a,b) in zip(alphas[i],betas[i]) ]
#         results.append(result)
        print()
        print(' '.join([ str(round(a, 2)) for a in result ]))
#     for i in range(m):
#         print(sum(column(results, i)))
#         print(column(betas, i))
        
def column(a, j):
    col = []
    m = len(a)
    for i in range(m):
        col.append(a[i][j])
    return col

def dot(x, y):
    return sum(a * b for (a, b) in zip(x, y))

def product(x, y):
    return [a * b for (a, b) in zip(x, y)]

результаты

In [2]:
import copy

def check():
    test_number = input()
    i = 0
    if test_number not in ['1', '2']:
        print('wrong test number')
    else:
        if test_number == '2':
            i = 1
        t = copy.deepcopy(tests)
        print()
        print(viterbi(t[i]['p'], t[i]['a'], t[i]['b'], t[i]['seq_open']))
        forward_backward(t[i]['p'], t[i]['a'], t[i]['b'], t[i]['seq_open'])

In [122]:
check()

1

11111111222222222211111111

0.86 0.77 0.88 0.78 0.89 0.8 0.91 0.86 0.46 0.27 0.19 0.15 0.14 0.14 0.15 0.19 0.28 0.47 0.89 0.96 0.98 0.98 0.98 0.98 0.97 0.94

0.14 0.23 0.12 0.22 0.11 0.2 0.09 0.14 0.54 0.73 0.81 0.85 0.86 0.86 0.85 0.81 0.72 0.53 0.11 0.04 0.02 0.02 0.02 0.02 0.03 0.06


In [121]:
check()

2

21212122111111111122222222

0.5 0.51 0.5 0.51 0.5 0.51 0.5 0.5 0.51 0.51 0.51 0.51 0.51 0.51 0.51 0.51 0.51 0.51 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5

0.5 0.49 0.5 0.49 0.5 0.49 0.5 0.5 0.49 0.49 0.49 0.49 0.49 0.49 0.49 0.49 0.49 0.49 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
